In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer, EncoderDecoderModel 
tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install python-telegram-bot --upgrade


In [ ]:
import logging
from telegram import Update
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger()

def get_length_param(text: str) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')
   
def answer(update: Update, context: CallbackContext):
    global step, chat_history_ids

    input_user = update.message.text   #input("===> User:")

    # encode the new user input, add parameters and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(f"|0|{get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|", return_tensors="pt")

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response
    chat_history_ids = model.generate(
        bot_input_ids,
        num_return_sequences=1,
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature = 0.6,
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
        device='cpu',
    )

    update.message.text = f"AnBot: {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)}"
    update.message.reply_text(update.message.text)    
    
    step = step+1 
    if step == 10:
        step = 0
    
updater = Updater('1967613159:AAG_c5C6a2spE_QpcG0ms658J4kbCjYzr2s', use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, answer))

# Start the Bot

step = 0

updater.start_polling()
updater.idle()

2021-08-30 06:45:16,857 - apscheduler.scheduler - INFO - Scheduler started
2021-08-30 06:53:51,586 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-08-30 06:53:51,589 - apscheduler.scheduler - INFO - Scheduler has been shut down
